# Basic RNA circuit simulation 



In [2]:
from bioreaction.simulation.basic_sim import convert_model

import jax
import numpy as np
import os
import sys

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-paper')
os.environ["TF_CPP_MIN_LOG_LOVEL"] = "0"
jax.config.update('jax_platform_name', 'gpu')


if __package__ is None:

    module_path = os.path.abspath(os.path.join('..'))
    sys.path.append(module_path)

    __package__ = os.path.basename(module_path)


from src.utils.data.data_format_tools.common import load_json_as_dict
from src.utils.common.setup_new import prepare_config, construct_circuit_from_cfg
from src.utils.circuit.agnostic_circuits.circuit_manager_new import CircuitModeller
from tests_local.shared import five_circuits

config = load_json_as_dict('../tests_local/configs/simple_circuit.json')


In [3]:



def update_model_rates(model, a=None, d=None, ka=None, kd=None):
    for i, r in enumerate(model.reactions):
        if not r.input:  # 0 -> RNA
            if a is not None:
                model.reactions[i].forward_rate = a[model.species.index(
                    r.output[0])]
                model.reactions[i].reverse_rate = 0
        elif not r.output:  # RNA -> 0
            if d is not None:
                model.reactions[i].forward_rate = d[model.species.index(
                    r.input[0])]
                model.reactions[i].reverse_rate = 0
        else: # unbound RNA -> bound RNA
            if ka is not None:
                model.reactions[i].forward_rate = ka[model.species.index(r.input[0]),
                                                     model.species.index(r.input[1])]
            if kd is not None:
                model.reactions[i].reverse_rate = kd[model.species.index(r.input[0]),
                                                     model.species.index(r.input[1])]
    return model


def make_params(model, scale_rates=True):
    sim_model = convert_model(model)

    if scale_rates:
        m = np.max([sim_model.forward_rates.max(), sim_model.reverse_rates.max()])
    else:
        m = 1
    inputs = sim_model.inputs
    outputs = sim_model.outputs
    forward_rates = sim_model.forward_rates/m
    reverse_rates = sim_model.reverse_rates/m
    return inputs, outputs, forward_rates, reverse_rates, m

In [5]:
config['simulation']

{'dt': 0.1,
 't0': 0,
 't1': 100,
 'tmax': 20000,
 'solver': 'diffrax',
 'use_batch_mutations': True,
 'batch_size': 100,
 'max_circuits': 1000,
 'device': 'cpu',
 'threshold_steady_states': 0.1,
 'use_rate_scaling': True}

Use the 5 tester circuits :)

In [8]:
config = prepare_config(config)
circuits, config, result_writer = five_circuits(config)

# config['interactions'] = {}
config['interactions'] = None
tester_circuit = construct_circuit_from_cfg({'data_path': '../tester.fasta'}, config)
circuits.append(tester_circuit)


In [ ]:
tester_circuit.interactions_state

'uninitialised'

In [ ]:

circuit_modeller = CircuitModeller(result_writer=result_writer, config=config)
circuits = circuit_modeller.batch_circuits(
    circuits=circuits,
    methods={
        "compute_interactions": {},
        "init_circuits": {'batch': True},
        'simulate_signal_batch': {'ref_circuit': None,
                                  'batch': config['simulation']['use_batch_mutations']},
        'write_results': {'no_visualisations': False, # config['experiment']['no_visualisations'],
                          'no_numerical': False} #config['experiment']['no_numerical']}
    }
)


simulator.py:check_IntaRNA_path():115: Could not detect IntaRNA on system: CompletedProcess(args='which IntaRNA', returncode=1, stdout='', stderr='') WARNING
circuit_manager_new.py:prepare_internal_funcs():467: Signal differs between circuits, but only first signal used for simulation. WARNING
circuit_manager_new.py:batch_circuits():537: 	From 6 circuits, a total of 6 mutated circuits will be simulated. WARNING
circuit_manager_new.py:batch_circuits():544: 		Starting new round of viable circuits (0 - 6 / 6) WARNING
circuit_manager_new.py:batch_circuits():562: 		Making subcircuits 0 - 6 took 1.2e-05s WARNING
circuit_manager_new.py:batch_circuits():568: 	Batching 0 - 6 circuits (out of 0 - 6 (total: 6)) (Circuits: 0 - 6 of 6) WARNING
circuit_manager_new.py:run_batch():599: 		Running 6 Subcircuits - 0_weak: compute_interactions WARNING


FileNotFoundError: [Errno 2] No such file or directory: 'IntaRNA'